In [ ]:
! pip install accelerate peft bitsandbytes git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-3t79w2cr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-3t79w2cr
  Resolved https://github.com/huggingface/transformers to commit 9fd606dbdb392e76e66a4e4e852948ebf814dbb6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417

In [ ]:
import torch
# from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer
from trl import SFTTrainer
import os
import pandas as pd

In [ ]:
data = pd.read_csv('/content/image.csv')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
data

,Medicine Name,Composition,Uses,Side_effects,Unnamed: 4,Manufacturer,Excellent Review %,Average Review %,Poor Review %
0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,NaN,Roche Products India Pvt Ltd,22,56,22
1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...,NaN,Glaxo SmithKline Pharmaceuticals Ltd,47,35,18
2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea,NaN,Alembic Pharmaceuticals Ltd,39,40,21
3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...,NaN,Glenmark Pharmaceuticals Ltd,24,41,35
4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance,NaN,Cadila Pharmaceuticals Ltd,34,37,29
...,...,...,...,...,...,...,...,...,...
11820,Zilarta-CT 40/6.25 Tablet,Azilsartan medoxomil (40mg) + Chlorthalidone (...,Hypertension (high blood pressure),Dizziness Tiredness Decreased potassium level ...,NaN,Micro Labs Ltd,67,33,0
11821,Zipcoz Tablet,Chromium Picolinate (100mcg) + D-Chiro Inosito...,Polycystic ovarian syndrome (PCOS),Rash Nausea Paresthesia tingling or pricking s...,NaN,Lifeline Remedies India Pvt Ltd,0,80,20
11822,Zestasil 100 Tablet,Sildenafil (100mg),Treatment of Erectile dysfunction,Flushing sense of warmth in the face ears neck...,NaN,TBG pharma ltd,25,75,0
11823,Zedruff Shampoo,Ketoconazole (2% w/v) + Zinc pyrithione (1% w/v),Treatment of Dandruff,Application site reactions burning irritation ...,NaN,Minova Life Sciences Pvt Ltd,0,0,100


In [ ]:
data.columns

Index(['Medicine Name', 'Composition', 'Uses', 'Side_effects', 'Unnamed: 4',
       'Manufacturer', 'Excellent Review %', 'Average Review %',
       'Poor Review %'],
      dtype='object')

In [ ]:
data.drop(columns=['Unnamed: 4','Excellent Review %','Average Review %','Poor Review %'],inplace=True,axis=1)

In [ ]:
data.head()


,Medicine Name,Composition,Uses,Side_effects,Manufacturer
0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,Roche Products India Pvt Ltd
1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...,Glaxo SmithKline Pharmaceuticals Ltd
2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea,Alembic Pharmaceuticals Ltd
3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...,Glenmark Pharmaceuticals Ltd
4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance,Cadila Pharmaceuticals Ltd


In [ ]:
data.isnull().sum()

Medicine Name    0
Composition      0
Uses             0
Side_effects     0
Manufacturer     0
dtype: int64

In [ ]:
data.columns


Index(['Medicine Name', 'Composition', 'Uses', 'Side_effects', 'Manufacturer'], dtype='object')

In [ ]:

def generate_text(medicine_name, uses, side_effects, composition,Manufacturer):
    return f"###Human : {medicine_name}\n ###Assistant : Uses: {uses}\n Side Effects : {side_effects}\n Composition: {composition}\n Manufacturer : {Manufacturer}\n"


data["text"] = data.apply(lambda row: generate_text(row["Medicine Name"], row["Uses"], row["Side_effects"], row["Composition"],row['Manufacturer']), axis=1)

print(data)


                   Medicine Name  \
0        Avastin 400mg Injection   
1       Augmentin 625 Duo Tablet   
2            Azithral 500 Tablet   
3               Ascoril LS Syrup   
4              Aciloc 150 Tablet   
...                          ...   
11820  Zilarta-CT 40/6.25 Tablet   
11821              Zipcoz Tablet   
11822        Zestasil 100 Tablet   
11823            Zedruff Shampoo   
11824            Zedruff Shampoo   

                                             Composition  \
0                                    Bevacizumab (400mg)   
1        Amoxycillin  (500mg) +  Clavulanic Acid (125mg)   
2                                   Azithromycin (500mg)   
3      Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...   
4                                     Ranitidine (150mg)   
...                                                  ...   
11820  Azilsartan medoxomil (40mg) + Chlorthalidone (...   
11821  Chromium Picolinate (100mcg) + D-Chiro Inosito...   
11822                      

In [ ]:
from datasets import load_dataset, Dataset
data = Dataset.from_pandas(data)


In [ ]:
data

Dataset({
    features: ['Medicine Name', 'Composition', 'Uses', 'Side_effects', 'Manufacturer', 'text'],
    num_rows: 11825
})

In [ ]:
data[5]

{'Medicine Name': 'Allegra 120mg Tablet',
 'Composition': 'Fexofenadine (120mg)',
 'Uses': 'Treatment of Sneezing and runny nose due to allergiesTreatment of Allergic conditions',
 'Side_effects': 'Headache Drowsiness Dizziness Nausea',
 'Manufacturer': 'Sanofi India Ltd',
 'text': '###Human : Allegra 120mg Tablet\n ###Assistant : Uses: Treatment of Sneezing and runny nose due to allergiesTreatment of Allergic conditions\n Side Effects : Headache Drowsiness Dizziness Nausea\n Composition: Fexofenadine (120mg)\n Manufacturer : Sanofi India Ltd\n'}

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("Varshitha/flan-t5-large-finetune-medicine-v5")
model = T5ForConditionalGeneration.from_pretrained("Varshitha/flan-t5-large-finetune-medicine-v5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:

training_args = TrainingArguments(
    output_dir="medflan-t5-large",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=5e-6,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=5,
    num_train_epochs=20,
    max_steps=250,
    fp16=True,
)


trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        dataset_text_field="text",
        args=training_args,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
)

trainer.train()